In [3]:
from speech_to_text import SpeechToText
stt = SpeechToText()


/home/duy1332002/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
stt.print_result("/home/duy1332002/Desktop/Lip_Reading_THDH/data/speech_to_text/test7.wav")

 cho đến một ngày khi ngồi từ tốn viết xuống một thứ gì đó gọi là


In [4]:
stt.save_result_to_file('/home/duy1332002/Desktop/Lip_Reading_THDH/data/speechtotext/t5.wav', '/home/duy1332002/Desktop/Lip_Reading_THDH/data/speechtotext/test1.txt')

Save result to file:  /home/duy1332002/Desktop/Lip_Reading_THDH/data/speechtotext/test1.txt


# Correction

In [4]:
import soundfile as sf
import pandas as pd
import torch
import numpy as np
import IPython
import pvleopard
from underthesea import word_tokenize

In [22]:
def check_alignment(audio_path, alignment_path, save_audio=False, path_save_audio=None):
    '''
    check the result of alignment can be predict by speech to text or not
    parameter:
        audio_path: str - path audio
        alignment_path: str - path csv
        save_audio: bool - save audio or not
        path_save_audio: str - path save audio, if save_audio is True, you should give the path
    return:
        pandas table - the alignment is correct or not
    '''
    # check the audio file is exist or not
    if not os.path.exists(audio_path):
        print('File not found')
        return None
    # load dummy dataset and read soundfiles
    ds = stt.map_to_array({"file": audio_path})
    ds['speech'] = np.mean(ds['speech'], axis = 1)

    # read alignment csv
    df_alignment = pd.read_csv(alignment_path, names=['start', 'end', 'word'])
    df_result = pd.DataFrame(columns=['start', 'end', 'word', 'predict'])

    t = 1 / ds['sampling_rate']

    audio_list =  []
    for i in range(len(df_alignment)):
        start = int((df_alignment['start'].values[i] / t))
        end = int((df_alignment['end'].values[i] / t))
        word_sound = ds['speech'][start:end]
        word_predict = stt._transcribe_with_vector(word_sound, ds)
        if save_audio:
            if not os.path.exists(path_save_audio):
                os.mkdir(path_save_audio)
            sf.write(path_save_audio + '/' + df_alignment['word'].values[i] + '.wav', word_sound, ds['sampling_rate'])
        df_result = pd.concat([df_result, pd.DataFrame([[df_alignment['start'].values[i], 
        df_alignment['end'].values[i], df_alignment['word'].values[i], word_predict]], 
        columns=['start', 'end', 'word', 'predict'])], ignore_index=True)
    return df_result

In [23]:
k = check_alignment('test7.wav', 'test7.csv', save_audio=True, path_save_audio='test7')

In [24]:
k

,start,end,word,predict
0,0.336,0.528,cho,
1,0.624,0.720,đến,
2,0.768,1.008,một,hai
3,1.056,1.248,ngày,
4,1.296,1.440,khi,
5,1.488,1.680,ngồi,hồi
6,1.728,1.920,từ,
7,1.968,2.208,tốn,
8,2.256,2.496,viết,
9,2.544,2.784,xuống,


In [27]:
def second_to_timecode(x: float) -> str:
    hour, x = divmod(x, 3600)
    minute, x = divmod(x, 60)
    second, x = divmod(x, 1)
    millisecond = int(x * 1000.)
    return '%.2d:%.2d:%.2d,%.3d' % (hour, minute, second, millisecond)

In [29]:
def convert_to_srt(alignement_path):
    df = pd.read_csv(alignement_path, names=['start', 'end', 'word'])
    with open('test.srt', 'w') as f:
        for i in range(len(df)):
            f.write(f'{i+1}\n')
            f.write(f'{second_to_timecode(df["start"].values[i])} --> {second_to_timecode(df["end"].values[i])}\n')
            f.write(f'{df["word"].values[i]}\n\n')
convert_to_srt('test7.csv')
